In [1]:
from bs4 import BeautifulSoup
import lxml
import time
import datetime
from datetime import datetime
from tqdm import tqdm 
import csv
import os 
import pandas as pd
import numpy as np
import requests
import codecs



In [30]:
animeTitle = []
animeType = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = []



In [31]:
#for j in range(1,384):
for i in range(1,51):
        # give the file paths of your local folder
        path = "/home/sssiyam/ADM-HW3/html/page_1" + "/article_"+ str(i) + ".html"

        file = codecs.open(path, "r", "utf-8")
        soup = BeautifulSoup(file , 'html.parser')

        #getting the anime title
        try:
            animeTitle.append(soup.find_all('strong')[0].contents[0])
        except: 
            animeTitle.append('NA')
        
        #getting the date
            if span.contents[0] == 'Aired:':
                try:
                    if len(div.contents[2]) > 21:
                        release = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        releaseDate.append(release)
                        end = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        endDate.append(end)
                    else:
                        release = pd.to_datetime(div.contents[2][1:16]).to_pydatetime().strftime('%m/%d/%Y')
                        releaseDate.append(release)
                        endDate.append('-')
                except:
                        releaseDate.append('NA')
                        endDate.append('NA')
        
        divs = soup.find_all("div", {"class": "spaceit_pad"})
        for div in divs:
            spans = div.find_all("span")
            for span in spans:

                #getting the anime types
                try:
                    if span.contents[0] == 'Type:':
                        animeType.append(div.find_all('a')[0].contents[0])
                except:
                    animeType.append('NA')

                # number of episodes
                if span.contents[0] == 'Episodes:':
                    try: 
                        animeNumEpisode.append(int(div.contents[2]))
                    except:
                        animeNumEpisode.append(0)


        divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
        for div in divs:
             # getting the anime members
            try:
                members = div.find_all("span", {"class": "numbers members"})
                animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
            except: 
                animeNumMembers.append(0)

            # getting the users
            users = div.find_all("div", {"class": "fl-l score"})
            # here we we eliminate the word 'user '   
            # that is why there is the [:-6] part
            # we also replace the comma divisor
            try:
                animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
            except:
                animeUsers.append(0)

            # getting the scores
            rating=soup.find(name="div",attrs={"class":"fl-l score"})
            try:        
                animeScore.append(float(rating.text.strip()))
            except:
                animeScore.append(None)
            
            # getting anime ranks
            try:
                rank = div.find_all("span", {"class": "numbers ranked"})
                animeRank.append(int(rank[0].contents[1].contents[0][1:]))
            except:
                animeRank.append(None)

            # getting the popularity
            try:   
                popularity = div.find_all("span", {"class": "numbers popularity"})
                animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
            except:
                animePopularity.append(None)

            #getting the synopsis 
            try:
                description = soup.find_all("p", {"itemprop": "description"})
                for br in description[0].find_all("br"):
                    br.replace_with("\n")
                animeDescription.append(description[0].contents)
            except: 
                animeDescription.append('NA')

        
            # geting the related animes
            try:
                related = soup.find_all("table", {"class": "anime_detail_related_anime"})
                x = []
                y = []
                for tr in related:
                    td = tr.find_all("td")
                    for i in range(0, len(td), 2):
                        x.append(td[i].contents[0])
                        try:
                            t = td[i+1].find_all("a")
                            y.append(t[0].contents[0])
                        except:
                            y.append('NA')

                    animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
            except: 
                animeRelated.append('NA')
             #getting the charecters
            try:
                characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
                chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
                x = []
                for i in chars:
                    x.append(i.contents[0].contents[0])
                animeCharacters.append(x)
            except:
                animeCharacters.append("NA")

            # getting the voices
            try:
                voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
                y = []
                for i in voices:
                    y.append(i.contents[1].contents[0])
                animeVoices.append(y)
            except:
                animeVoices.append("NA")

            #getting the saff of the anime
            try:
                staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
                staff = staff[1].find_all("td")
                x = []
                y = []
                for i in range(1, len(staff), 2):
                    x.append(staff[i].contents[1].contents[0])
                    y.append(staff[i].find_all("small")[0].contents[0])
                animeStaff.append([list(i) for i in list(zip(x,y))])
            except:
                animeStaff.append("NA")



In [40]:
dataset = pd.DataFrame(
    [animeTitle, animeType, animeNumEpisode, animeNumMembers, 
     animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, 
     animeCharacters, animeVoices, animeStaff], 
    index=['animeTitle', 'animeType', 'NumOfEpisodes', 'animeMembers', 'animeScore', 
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', 'animeVoices', 'animeStaff']).T

In [35]:
(releaseDate)

[]

In [41]:
dataset

,animeTitle,animeType,NumOfEpisodes,animeMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
0,Fullmetal Alchemist: Brotherhood,TV,64,2672149,9.16,1622384,1,3,[After a horrific alchemy experiment goes wron...,"[Adaptation: Fullmetal Alchemist, Alternative ...","[Elric, Edward, Elric, Alphonse, Mustang, Roy,...","[Park, Romi, Kugimiya, Rie, Miki, Shinichiro, ...","[[Cook, Justin, Producer], [Yonai, Noritomo, P..."
1,Gintama°,TV,51,483009,9.09,169476,2,337,"[Gintoki, Shinpachi, and Kagura return as the ...","[Adaptation: Gintama, Prequel: Gintama Movie 2...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[[Fujita, Youichi, Director, Storyboard, Plann..."
2,Shingeki no Kyojin Season 3 Part 2,TV,10,1592126,9.09,1087519,3,33,[Seeking to restore humanity's diminishing hop...,"[Adaptation: Shingeki no Kyojin, Prequel: Shin...","[Levi, Ackerman, Mikasa, Yeager, Eren, Arlert,...","[Kamiya, Hiroshi, Ishikawa, Yui, Kaji, Yuki, I...","[[Yabuta, Shuuhei, Producer], [Wada, Jouji, Pr..."
3,Steins;Gate,TV,24,2088500,9.09,1109700,4,11,[The self-proclaimed mad scientist Rintarou Ok...,"[Adaptation: Steins;Gate, Alternative setting:...","[Okabe, Rintarou, Makise, Kurisu, Shiina, Mayu...","[Miyano, Mamoru, Imai, Asami, Hanazawa, Kana, ...","[[Iwasa, Gaku, Producer], [Yasuda, Takeshi, Pr..."
4,Fruits Basket: The Final,TV,13,274006,9.07,113310,5,651,"[Hundreds of years ago, the Chinese Zodiac spi...","[Adaptation: Fruits Basket, Prequel: Fruits Ba...","[Souma, Kyou, Honda, Tooru, Souma, Yuki, Souma...","[Uchida, Yuuma, Iwami, Manaka, Shimazaki, Nobu...","[[Ibata, Yoshihide, Director], [Aketagawa, Jin..."
5,Gintama',TV,51,428347,9.06,178766,6,379,"[After a one-year hiatus, Shinpachi Shimura re...","[Adaptation: Gintama, Prequel: Gintama, Charac...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[[Fujita, Youichi, Director, Episode Director,..."
6,Hunter x Hunter (2011),TV,148,2146315,9.06,1252938,7,10,"[[Hunter x Hunter], is set in a world where H...","[Adaptation: Hunter x Hunter, Alternative vers...","[Zoldyck, Killua, Kurapika, Freecss, Gon, Pala...","[Ise, Mariya, Sawashiro, Miyuki, Han, Megumi, ...","[[Maruyama, Masao, Producer], [Nakatani, Toshi..."
7,Ginga Eiyuu Densetsu,OVA,110,262688,9.04,58459,8,674,[The 150-year-long stalemate between the two i...,"[Adaptation: Ginga Eiyuu Densetsu, Prequel: Gi...","[Yang, Wen-li, von Lohengramm, Reinhard, von R...","[Tomiyama, Kei, Horikawa, Ryo, Wakamoto, Norio...","[[Ishiguro, Noboru, Director, Episode Director..."
8,Gintama': Enchousen,TV,13,260726,9.04,130041,9,681,"[While Gintoki Sakata was away, the Yorozuya f...","[Adaptation: Gintama, Prequel: Gintama', Seque...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Kugimiya, Rie, Sakaguchi, D...","[[Fujita, Youichi, Director, Episode Director,..."
9,Gintama: The Final,Movie,1,79027,9,29979,10,1924,"[New , [Gintama], movie.]","[Adaptation: Gintama, Prequel: Gintama.: Shiro...","[Sakata, Gintoki, Kagura, Shimura, Shinpachi, ...","[Sugita, Tomokazu, Ishida, Akira, Hino, Satoshi]","[[Fujita, Youichi, Director], [Miyawaki, Chizu..."
